![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with Redispy
## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/00_redispy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Prepare data

In this examples we will load a list of movie objects with the following attributes: `title`, `rating`, `description`, and `genre`. 

For the vector part of our vector search we will embed the description so that user's can search for movies that best match what they're looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [5]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 204 (delta 37), reused 24 (delta 24), pack-reused 152
Receiving objects: 100% (204/204), 9.47 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (64/64), done.
mv: temp_repo/python-recipes/vector-search/resources: No such file or directory


## Packages

In [24]:
%pip install -q "redis>=5.0.5" numpy sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [1]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [2]:
from redis import Redis
client = Redis.from_url(REDIS_URL)
client.ping()

True

In [3]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer

# load model for embedding our movie descriptions
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(model, text):
    return np.array(model.encode(text)).astype(np.float32).tobytes()

/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Note: convert embedding array to bytes for storage in Redis Hash data type
movie_data = [
    {
        **movie,
        "vector": embed_text(model, movie["description"])
    } for movie in movies
]

In [6]:
movie_data[0]

{'id': 1,
 'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'vector': b'\x8bf|=\xc3`\n;\xf2\x91\xb7;?\xcb~\xbd\xdfd\xce\xbb\xc7\x16J=H\xa7?=\xdfv\x95<h\xfa\x06\xbe\x11Y\xcf=)\x07p=E\xdb\r\xbd\x93\xf2H\xbdke\xc6<@\xdfa=o8\x16\xbc\xf1\xd3\x13<8\xaa\x1c=\x14\xef\x89<\xc1\xb0-<\x9d\xb2\x9f\xbc^\x0b\xc3\xbd\xa5NR=ol\xf7\xbcP>\x17\xbeA\x1e\x05\xb9Hu\xbf<B\xe3b\xba\xd8\xa6\xa8\xbd\x98\xdc\xec\xbc`c%=\x81\xe7r\xbb$OG=:(\x85=a@\xa2\xbc-Z\xd0\xbdB%K\xbd\xc8\xed\x94\xbcW\xddH=\x8e&F<\xde*\xec<\x8d\xd8\x8d\xbd\xbdZ\x98<\x14\xa3\xa3=>g3\xbd$\xcd\xbd\xbd\xa1$\xf7;\x04\xf5z=\xfc\xb4\x8c=\x89\x0e\xc6\xbdhI\x90\xbd^\x16\xbd;z\xe7\x0c\xbd\x1b3\xc9\xbc\x89\xf8\xbb\xbc\x18\'u\xbb>\x8f\xca<\x02\x80J=\x0e\xaf*=\x8dOU\xbd\xcf\xf0\x95\xbc \x02\x19=\x19\xf4K<\xc5\xc2\t=J\x83\xac=\x95\xd7\xb8\xbd\xf2\xb5\x9c\xbd=\x85\x18=\x94d&=03\xf8<\xee\xf7\x88<\x80v\xf2\xbb9=[\xbdG\xac\xee\xbb<

## Define Redis index schema

In [7]:
from redis.commands.search.field import VectorField, TagField, NumericField, TextField
from redis.commands.search.index_definition import IndexDefinition, IndexType

index_name = "movies"

schema = (
    VectorField(
        "vector",
        "HNSW",
        {
            "TYPE": "FLOAT32",
            "DIM": 384,
            "DISTANCE_METRIC": "COSINE"
        }
        ),
        NumericField("rating"),
        TagField("genre"),
        TextField("title"),
        TextField("description")
)

try:
    client.ft(index_name).info()
    print("Index exists!")
except:
    # index Definition
    definition = IndexDefinition(index_type=IndexType.HASH)

    # create Index
    client.ft(index_name).create_index(fields=schema, definition=definition)

## Populate index

In [8]:
def load_docs(client: Redis, data: list[dict]):
    for i, d in enumerate(data):
        client.hset(
            i,
            mapping = d
        )

def print_results(res):
    docs = [(doc.title, doc.genre, doc.rating) for doc in res.docs]
    print(f"Top {len(docs)} movies: ", docs)

In [9]:
load_docs(client, movie_data)

In [10]:
res = client.ft(index_name).search("*")
res

Result{20 total, docs: [Document {'id': '0', 'payload': None, 'genre': 'action', 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.', 'vector': 'f|=`\n;;?~dλ\x16J=H?=v<h\x06\x11Y=)\x07p=E\rHke<@a=o8\x16\x13<8\x1c=\x14<-<^\x0býNR=olP>\x17A\x1e\x05Hu<Bbئ`c%=r$OG=:(=a@-ZнB%K피WH=&F<*<؍Z<\x14=>g3$ͽ$;\x04z==\x0eƽhI^\x16;z\x0c\x1b3ɼ\x18\'u><\x02J=\x0e*=OU \x02\x19=\x19K<\t=J=\u05f8\U000b573d=\x18=d&=03<<v9=[G<:Ad\x19!d\x1dx;;O<1,쮮=r\x00-"\x06l\x1a=Ŀ=\x19\x150=\U0005df26GK= =I-e:\x1c=\x7f<n\x16\\p<<9s<=4&<\x1c\x18<\x18-<\x15N=/=\x1d<\x17L\x13<\x10\x10κcܼ8\x05=+<bR-\\91=aV=\x1eGּ"S=^<\x12\x13<y/\x06p<?Н<H\x02=\x1a\x14\U000a6efceQKH\x10/tLɹ\x0cik=\r<1f]=\x1a 4/vfXv=\x0c\x0cq<0\x1c\x08<nt<\x19=m\x1dm^\x1bV\x0emFU;G|Zv*=|\x10;\x08!\x0e\x00Դ=\x14&\x15R=d$:<1<"\x08\x00i<\x01<\x00=1b|!\x1cŹ>\x1bF?\x14ď(O<\x12=\x05Op\x18ɞeHs;$a;B#:=\x15*<\x1b=\x01V\r=9\x06=\x13u<\x1a;9/=\x85=p\x0b"=i<:c=2\x08\x12;=OVW;Vb<Н<r;qzV\x00<\x19<J%!翻\x7f\x12=\x1d=b

## Index loaded now we can perform vector search

### basic vector search

In [11]:
from redis.commands.search.query import Query

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: dialect 2 and above required for vector search
query = Query("(*)=>[KNN 3 @vector $vec_param AS dist]").sort_by("dist").dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)


Top 3 movies:  [('Fast & Furious 9', 'action', '6'), ('Despicable Me', 'comedy', '7'), ('The Incredibles', 'comedy', '8')]


### Hybrid filter vector search

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

In [12]:
# Search for top 3 movies specifically in the action genre

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: genre is a tag field in our schema so the syntax is @<field_name>:{ <tag> | <tag> | ...}
query = Query("(@genre:{action})=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Fast & Furious 9', 'action', '6'), ('Mad Max: Fury Road', 'action', '8'), ('Explosive Pursuit', 'action', '7')]


In [13]:
# Search for top 3 movies specifically in the action genre with ratings at or above a 7

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@genre:{action} & (@rating:[7 inf]))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Mad Max: Fury Road', 'action', '8'), ('Explosive Pursuit', 'action', '7'), ('The Avengers', 'action', '8')]


In [14]:
# Search with full text search for movies that directly mention "criminal mastermind" in the description

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@description:(criminal mastermind))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 2 movies:  [('Despicable Me', 'comedy', '7'), ('The Dark Knight', 'action', '9')]


In [15]:
# Vector search with wild card match

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

query = Query("(@description:(crim*))=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('Despicable Me', 'comedy', '7'), ('The Incredibles', 'comedy', '8'), ('Explosive Pursuit', 'action', '7')]


In [16]:
# Vector search with fuzzy match

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

# Note: fuzzy match is based on Levenshtein distance. Therefore, "hero" might return result for "her" as an example.
# See docs for more info https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/query_syntax/
query = Query("(@description:%hero%)=>[KNN 3 @vector $vec_param AS dist]").sort_by('dist').dialect(2)

res = client.ft(index_name).search(query, query_params = {'vec_param': embedded_user_query})

print_results(res)

Top 3 movies:  [('The Avengers', 'action', '8'), ('Black Widow', 'action', '7'), ('The Princess Diaries', 'comedy', '6')]


## Range queries

Range queries allow you to set a pre defined "threshold" for which we want to return documents. This is helpful when you only want documents with a certain distance from the search query.

In [17]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

query = (
    Query("@vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("title", "rating", "genre", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vector": embedded_user_query
}

res = client.ft(index_name).search(query, query_params)
print_results(res)


Top 6 movies:  [('The Incredibles', 'comedy', '8'), ('Black Widow', 'action', '7'), ('Despicable Me', 'comedy', '7'), ('Shrek', 'comedy', '8'), ('Monsters, Inc.', 'comedy', '8'), ('Aladdin', 'comedy', '8')]


Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `or` condition that returns vector search within the defined range or with a rating at or above 9.

In [18]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

query = (
    Query("@rating:[9 +inf] | @vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("title", "rating", "genre", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.7,
    "vector": embedded_user_query
}

res = client.ft(index_name).search(query, query_params)
print_results(res)

Top 3 movies:  [('The Incredibles', 'comedy', '8'), ('The Dark Knight', 'action', '9'), ('Inception', 'action', '9')]


### Additional queries

In addition to the variety of vector queries shown above redis supports full-text search, aggregations, and various weighting strategies that can be mixed and matched for a wide range of search applications.

### Full text search with BM25

The following query does a pure token based BM25 search with redis.

In [19]:
input = "Criminal mastermind"

# Redis breaks searches into key tokens
def tokenize(query):
    return " | ".join(query.split(" ")).lower()

user_query = Query(tokenize(input))\
                .scorer("BM25STD") \
                .with_scores() \
                .return_fields("title", "genre", "rating", "description") \
                .paging(0, 10) # limits the amount of results to 10

res = client.ft(index_name).search(user_query)
res.docs

[Document {'id': '6', 'payload': None, 'score': 6.267822483123378, 'title': 'The Dark Knight', 'genre': 'action', 'rating': '9', 'description': 'Batman faces off against the Joker, a criminal mastermind who threatens to plunge Gotham into chaos.'},
 Document {'id': '17', 'payload': None, 'score': 5.846220066150412, 'title': 'Despicable Me', 'genre': 'comedy', 'rating': '7', 'description': 'When a criminal mastermind uses a trio of orphan girls as pawns for a grand scheme, he finds their love is profoundly changing him for the better.'},
 Document {'id': '0', 'payload': None, 'score': 3.9323951881774195, 'title': 'Explosive Pursuit', 'genre': 'action', 'rating': '7', 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.'}]

### Weighting (boosting)

Sometimes you might want a search to lean more heavily towards one condition over another and weight it higher in the result set.

In this example, you can see that even though `The Incredibles` isn't an `action` movie it is still the top result because it ranks highly on the fuzzy search for `%superhero%`.

In [20]:
query = Query('((@genre:{action}=>{$weight: 1}) | (@description:(%superhero%)=>{$weight: 10}))') \
        .return_fields("title", "genre", "rating", "description") \
        .paging(0, 3) \
        .dialect(2)

res = client.ft(index_name).search(query)
res.docs

[Document {'id': '15', 'payload': None, 'title': 'The Incredibles', 'genre': 'comedy', 'rating': '8', 'description': "A family of undercover superheroes, while trying to live the quiet suburban life, are forced into action to save the world. Bob Parr (Mr. Incredible) and his wife Helen (Elastigirl) were among the world's greatest crime fighters, but now they must assume civilian identities and retreat to the suburbs to live a 'normal' life with their three children. However, the family's desire to help the world pulls them back into action when they face a new and dangerous enemy."},
 Document {'id': '0', 'payload': None, 'title': 'Explosive Pursuit', 'genre': 'action', 'rating': '7', 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.'},
 Document {'id': '1', 'payload': None, 'title': 'Skyfall', 'genre': 'action', 'rating': '8', 'description': 'James Bond returns to track down a dangerous new enemy who threatens global secur

In [21]:
# clean up!
client.flushall()

True